In [28]:
import torch
import torchvision
import torchvision.transforms as transforms
import lpips
import time
import dnnlib
import pickle
import piqa

from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity
lpips2 = LearnedPerceptualImagePatchSimilarity(net_type='vgg')

# Define the batch size for loading data in mini-batches
batch_size = 64

# Transforms to apply to the data
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # Normalize the pixel values to [-1, 1]
    transforms.Resize(256),
])

def open_file_or_url(file_or_url):
    if dnnlib.util.is_url(file_or_url):
        return dnnlib.util.open_url(file_or_url, cache_dir='.stylegan2-cache')
    return open(file_or_url, 'rb')

def load_pkl(file_or_url):
    with open_file_or_url(file_or_url) as file:
        return pickle.load(file, encoding='latin1')

# Download and load the training set
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Download and load the test set
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

train_iterator = iter(train_loader)
images, labels = next(train_iterator)
images2, labels2 = next(train_iterator)

start = time.time()
loss_fn_alex = lpips.LPIPS(net = 'alex', verbose = False)
dist = loss_fn_alex(images, images2)
print(dist.shape)
print(time.time() - start)

start = time.time()
dist2 = lpips2(images, images2)
print(dist.shape)
print(time.time() - start)

# distance_measure = load_pkl('https://nvlabs-fi-cdn.nvidia.com/stylegan/networks/metrics/vgg16_zhang_perceptual.pkl')
# dist = distance_measure.get_output_for(images, images2)
# dist = (lpips_t0 - lpips_t1).square().sum(1) / epsilon ** 2


Files already downloaded and verified
Files already downloaded and verified
torch.Size([64, 1, 1, 1])
0.8282437324523926
torch.Size([64, 1, 1, 1])
10.823099136352539
